In [1]:
from datasets import load_dataset, DatasetDict, Dataset

ModuleNotFoundError: No module named 'datasets'

In [3]:
!pip install evaluate

  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl.metadata (18 kB)
  Using cached xxhash-3.4.1-cp38-cp38-win_amd64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py38-none-any.whl.metadata (7.1 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached aiohttp-3.9.3-cp38-cp38-win_amd64.whl.metadata (7.6 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp38-cp38-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp38-cp38-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp38-cp38-win_amd64.whl.metadata (32 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/84.1 kB ? eta -:--:--
   ---------------------------------------- 84.1/84.1 kB 2.3 MB/s eta 0:00:00
Using cached datasets-2.18.0

In [2]:
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

In [4]:
! pip install peft
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

  Using cached peft-0.9.0-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.9.0-py3-none-any.whl (190 kB)


In [5]:
model_checkpoint = 'LiyaT3/sentiment-analysis-imdb-distilbert'

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)
for name, param in model.named_parameters():
    param.requires_grad = False
# num=2
# model.layers[-1].trainable= True
# model.layers[-1].units= num

In [6]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [7]:
for name, param in model.named_parameters():
     if name.startswith("classifier"): # choose whatever you like here
        param.requires_grad = True

In [8]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_assisted_decoding',
 '_auto_class',
 '_autoset_attn_implementation',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_beam_sample',
 '_beam_search',
 '_buffers',
 '_call_impl',
 '_check_and_enable_flash_attn_2',
 '_check_and_enable_sdpa',
 '_compiled_call_impl',
 '_constrained_beam_search',
 '_contrastive_search',
 '_convert_head_mask_to_5d',
 '_copy_lm_head_original_to_resized',
 '_create_repo',
 '_dispatch_accelerate_model',
 '_expand_inputs_for_generation',
 

In [9]:
from datasets import load_dataset

dataset = load_dataset("sg247/binary-classification")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8004
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 2000
    })
})


In [10]:
import pandas as pd

df = pd.DataFrame(dataset['train'])
df.dropna(inplace = True)


df['label'] = df['label'].astype(int)
print(df)


                                                  tweet  label
0     Want to say a huge thanks to @WarriorAssaultS ...      1
1     @jaynehh_ you just need a job and get a letter...      1
2             @knhillrocks HA yes, make it quick tho :D      1
3     @shartyboy Thanks for texting me back :)) I'm ...      1
4     Laying out a greetings card range for print to...      1
...                                                 ...    ...
7999  "@aula_jr: @Jude_Mugabi I'm watching it. 2-0 M...      0
8000                                  why so sudden :((      0
8001        @21oclock :((( bout to instant transmission      0
8002  @ButtCupboard my feet were cold!!!! and there ...      0
8003  @Kimwoobin89__ oppa............./hug ur arm/ w...      0

[8000 rows x 2 columns]


In [11]:
d = pd.DataFrame(dataset['test'])
d.dropna(inplace = True)
d['label'] = d['label'].astype(int)


In [12]:
from datasets import Dataset, DatasetDict
train_dict = {
    'tweet': df['tweet'].tolist(),
    'label': df['label'].tolist()
}
train_dataset = Dataset.from_dict(train_dict)
dataset_dict = DatasetDict({'train': train_dataset})
print(dataset_dict['train']['label'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [13]:
from datasets import Dataset, DatasetDict
test_dict = {
    'tweet': d['tweet'].tolist(),
    'label': d['label'].tolist()
}
test_dataset = Dataset.from_dict(train_dict)
dataset_dict = DatasetDict({'test': train_dataset})

# print(dataset_dict['train']['label'][2])
print(dataset_dict['test']['label'][2])
print(dataset_dict)

1
DatasetDict({
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8000
    })
})


In [14]:
test_dict = {
    'tweet': d['tweet'].tolist(),
    'label': d['label'].tolist()
}

train_dict = {
    'tweet': df['tweet'].tolist(),
    'label': df['label'].tolist()
}



import datasets
train_dataset = datasets.Dataset.from_dict(train_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)
my_dataset_dict = datasets.DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})
print(my_dataset_dict)
print(my_dataset_dict['test']['label'][2])
print(my_dataset_dict['train']['label'][2])

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 2000
    })
})
1
1


In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

def tokenize_function(examples):
    tweets = examples["tweet"]
    preprocess_t= [tweet if isinstance(tweet , str) else str(tweet) for tweet in tweets]
    return tokenizer(preprocess_t , truncation= True , padding= "max_length")

tokenized_dataset = my_dataset_dict.map(tokenize_function, batched=True)
tokenized_dataset

Map: 100%|██████████| 2000/2000 [00:00<00:00, 5486.81 examples/s]


DatasetDict({
    train: Dataset({
        features: ['tweet', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['tweet', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [17]:
for name, param in model.named_parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
     if name.startswith("classifier"):
        param.requires_grad = True

In [21]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# training 

ranks = [4,100,20,1,10]
for r in ranks:
    peft_config = LoraConfig(task_type="SEQ_CLS",
                            r=r,
                            lora_alpha=32,
                            lora_dropout=0.01,
                            target_modules = ['q_lin','k_lin'])

    peft_config

    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    lr = 0.01
    batch_size = 32
    num_epochs = 1
    training_args = TrainingArguments(
        output_dir= model_checkpoint + "-lora-text-classification",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )
    accuracy = evaluate.load("accuracy")
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return accuracy.compute(predictions=predictions, references=labels)


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        eval_dataset = tokenized_dataset['train'],
        compute_metrics=compute_metrics, 
    )
    trainer.train()

trainable params: 665,858 || all params: 67,620,868 || trainable%: 0.9846930684178736


  0%|          | 0/250 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 6.18 GiB is allocated by PyTorch, and 22.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [39]:
trainer.save_model("Trained_Model_LoRA")

In [40]:
from transformers import AutoModelForSequenceClassification

path_to_saved_model = "Trained_Model_LoRA"
loaded_model = AutoModelForSequenceClassification.from_pretrained(path_to_saved_model)


In [48]:
inputs = tokenizer.encode("You are ugly", return_tensors="pt")
logits = loaded_model(inputs).logits
predictions = torch.max(logits,1).indices
print(predictions)
print("you are ugly" + " - " + id2label[predictions.tolist()[0]])

tensor([0])
you are ugly - Negative
